In [1]:
import numpy as np
import pandas as pd
import math

## 1. コーナー部の関数

### 1.1 エンドミル仕様の関数

In [2]:
def arc_contact(R, d ,T): #接触弧長さの関数
    r = d / 2 #エンドミル半径
    
    #計算用変数の定義
    A = R - r
    B = R ** 2 - r ** 2 + A ** 2 - 2 * T ** 2
    m = 1 + ((T + A) / T) ** 2
    n = 2 * A + B * (T + A) / T ** 2
    l = (A ** 2) + ((B ** 2) / (4 * T ** 2)) - (r ** 2)
    
    x = (- n + np.sqrt(n ** 2 - 4 * m * l)) / (2 * m) #交点のx座標
    
    #場合分け
    if x <= T:
        t = R + x
        L = r * np.arccos(1- t / r)
    
    else:
        alpha = A ** 2 + R ** 2 - 2 * R * T - r ** 2 + T ** 2
        X = A + np.sqrt(A ** 2 - alpha)
        t = R + X
        L = R * np.arccos(1 - (t / r))
    
    return L

In [3]:
def number_of_blades(R, d, n, T): #エンドミル刃数の関数(切削刃長さ一定の時)
    r = d / 2 #エンドミル半径
    L =  arc_contact(R, d ,T) #接触弧長さ
    pi = np.pi #円周率
    
    D = L / (n - 1) #切削刃長さ一定の条件
    z = 2 * pi * r / D #エンドミル刃数
    
    return z

In [4]:
def torsion_angle(d, U, z): #ねじれ角の関数
    pi = np.pi #円周率
    r = d / 2 #エンドミル半径
    
    theta = np.arctan(2 * pi * r / U / z) #ねじれ角
    theta = np.degrees(theta) #ラジアンをdegreeになおす
    
    return theta

In [5]:
def z_candidate(R, d, T, z_lim):#エンドミル刃数の候補を挙げる
    list_number_of_blades = []

    for n in range(2, z_lim, 1): #同時切削歯数を変化させたときのエンドミル刃数をそれぞれ求める
        z = number_of_blades(R, d, n, T) #刃数が少数を含んで出る
        z = np.round(z) #刃数を整数にする
        
        list_number_of_blades.append(z) #リストに格納
    
    list_number_of_blades = [i for i in list_number_of_blades if i <= z_lim] #算出された刃数のなかで最大刃数を超えるものを除外
    
    return list_number_of_blades

In [6]:
def depth_radious_true(R, d, T, z_lim): #刃数を整数にしたので実際の半径方向切込みを求める
    pi = np.pi #円周率
    r = d / 2 #エンドミル半径
    list_T_true = [] #格納用空リスト
    list_number_of_blades = z_candidate(R, d, T, z_lim) #刃数の候補を呼ぶ
    
    #計算用変数の定義
    A = R - r
    B = R ** 2 - r ** 2 + A ** 2 - 2 * T ** 2
    m = 1 + ((T + A) / T) ** 2
    n = 2 * A + B * (T + A) / T ** 2
    l = (A ** 2) + ((B ** 2) / (4 * T ** 2)) - (r ** 2)
    
    x = (- n + np.sqrt(n ** 2 - 4 * m * l)) / (2 * m) #交点のx座標
    
    for i in range(0, len(list_number_of_blades)): #刃数の候補それぞれに対応するaeを求める            
        D = 2 * pi * r / list_number_of_blades[i] #刃数に対応する刃の間隔
        L_true = (i + 1) * D #切削刃長さ一定となる接触弧
        
        #計算用変数
        X = np.abs(- (R - r) - r * np.cos(L_true / r)) 
        Y = r * np.sin(L_true / r)
        
        #交点のx座標による場合分け
        if x <= 0:
            T_true = (- (X + Y) + (np.sqrt((2 * (R ** 2)) - (X ** 2) - (Y ** 2) + (2 * X * Y)))) / 2
        
        elif x <= R - Y:
            T_true = (- (Y - X) + (np.sqrt((2 * (R ** 2)) - (X ** 2) - (Y ** 2) - (2 * X * Y)))) / 2
        
        else:
            T_true = R - r * np.sin(L_true / r)
        
        list_T_true.append(T_true) #リストに格納
    
    return list_T_true

In [7]:
def remove_zero(R, d, T, z_lim): #実際のaeが0になるものを除く
    list_number_of_blades = z_candidate(R, d, T, z_lim) #刃数の候補を呼ぶ
    list_T_true = depth_radious_true(R, d, T, z_lim) #aeの候補を呼ぶ
    
    #格納用空リスト
    list_number_of_blades_arg = []
    list_T_true_arg = []
    
    for i in range(0, len(list_number_of_blades)): #実際のaeが0になるものを除く
        
        if not list_T_true[i] == 0: #aeの候補i番目が0でなかったら
            list_number_of_blades_arg.append(list_number_of_blades[i]) #刃数の候補i番目の刃数を格納
            list_T_true_arg.append(list_T_true[i]) #aeの候補i番目のaeを格納
    
    return list_number_of_blades_arg, list_T_true_arg

In [8]:
def differ_calc(R, d, T, z_lim): #目標aeと実際のaeとの差
    list_differ = [] #格納用空リスト
    
    list_T_true = remove_zero(R, d, T, z_lim)[1] #aeの候補を呼ぶ
    
    for i in range(0, len(list_T_true)): #aeの候補それぞれに対応する目標aeとの差を求める
        differ = np.abs(T - list_T_true[i]) #差の絶対値
        list_differ.append(differ) #リストに格納
    
    return list_differ

In [9]:
def spec_endmill(R, d, ae, ap, z_lim): #エンドミル仕様を出力
    T = ae #以下aeをTとおく
    U = ap #以下apをUとおく
    
    if d / 2 > R: #エンドミル径が隅Rを超えている場合(計算不可)
        
        return print('エンドミル径が隅Rを超えてます')
    
    else: #計算可能な場合
        list_number_of_blades = z_candidate(R, d, T, z_lim) #刃数の候補を呼ぶ
        list_T_true =remove_zero(R, d, T, z_lim)[1] #実際のaeの候補を呼ぶ
    
    
    if len(list_number_of_blades) == 0: #刃数の候補が空だった場合(最大刃数を超える候補しかなかった場合)
        
        return print('最大刃数を超えました')
    
    elif len(list_T_true) == 0: #実際のaeが0になる場合(意味を持たない)
        
        return print('不可能な値です')
    
    else: #意味を持つ候補がある場合
        list_differ = differ_calc(R, d, T, z_lim) #目標aeと実aeの差を呼ぶ
        list_number_of_blades = remove_zero(R, d, T, z_lim)[0] #刃数の候補を呼ぶ
        
        n_min_differ = list_differ.index(min(list_differ)) #差が最小になるリスト番号を調べる
        
        z_dif = list_number_of_blades[n_min_differ] #差が最小になる刃数を抽出
        theta_dif = np.round(torsion_angle(d, U ,z_dif), decimals = 1) #ねじれ角を求める
        T_true = np.round(list_T_true[n_min_differ], decimals = 3) #差が最小になる実際のT
        
        D_dif = np.pi * d /z_dif
        L_true = arc_contact(R, d, T_true)
        n_dif = (L_true / D_dif) + 1
        n_dif = np.round(n_dif)
        
        #出力をまとめる
        end_mil = pd.DataFrame(
            data = np.array(
                [[z_dif, theta_dif, T_true, n_dif]]
            ),
            index = ['エンドミル仕様'],
            columns = ['刃数', 'ねじれ角', 'この仕様でのae', '同時切削刃数']
        )
        
        return end_mil

### 1.2 切込み深さの関数

In [10]:
def ae_calc(R, d, z, N):
    pi = np.pi
    r = d / 2
    D = 2 * pi * r / z

    L = (N - 1) * D
    X = np.abs(- (R - r) - r * np.cos(L / r))
    Y = r * np.sin(L / r)
    
    ae_1 = (- (X + Y) + (np.sqrt(np.abs((2 * (R ** 2)) - (X ** 2) - (Y ** 2) + (2 * X * Y))))) / 2
    ae_2 = (- (Y - X) + (np.sqrt(np.abs((2 * (R ** 2)) - (X ** 2) - (Y ** 2) - (2 * X * Y))))) / 2
    ae_3 = R - r * np.sin(L / r)
    
    list_ae = [ae_1, ae_2 ,ae_3]
    ae_candidate =[]
    
    if 0 in list_ae:
        list_ae.remove(0)
    
    for i in range(0,len(list_ae)):
        T = list_ae[i]
        A = R - r
        B = R ** 2 - r ** 2 + A ** 2 - 2 * T ** 2
        C = 1 + ((T + A) / T) ** 2
        D = 2 * A + B * (T + A) / T ** 2
        E = (A ** 2) + ((B ** 2) / (4 * T ** 2)) - (r ** 2)
        x = (- D + np.sqrt(np.abs(D ** 2 - 4 * C * E))) / (2 * C)
        
        if x <= 0:
            ae = ae_1
    
        elif x <= R - Y:
            ae = ae_2
        
        else:
            ae =ae_3
        
        ae_candidate.append(ae)
            
    ae = min(ae_candidate)

    return ae

In [11]:
def depth_cut(R, d, z, theta, m):
    ae_candidate = []
    
    for N in range(2, z):
        ae = ae_calc(R, d, z, N)
        ae_candidate.append(ae)
    
    if 0 in ae_candidate:
        ae_candidate.remove(0)
        
    ae = min(ae_candidate)
    
    pi = np.pi
    theta = np.radians(theta)
    r = d / 2
    
    D = 2 * pi * r / z
    L = arc_contact(R, d, ae)
    N = L / D
    N = np.round(N)
    ap = (D / np.tan(theta)) * m
    
    n = N + m
    
    ae = np.round(ae, decimals = 2)
    ap = np.round(ap, decimals = 2)
    
    depth_cut = pd.DataFrame(
        data = np.array([[ae, ap, n]]),
        index = ['切込み深さ'],
        columns = ['半径方向ae', '軸方向ap', '同時切削刃数']
    )

    return depth_cut

### 1.3 切削刃長さの関数

In [12]:
def length_blades_contact(R, d, z, theta, T, U ,u):
    pi = np.pi
    r = d / 2
    theta = np.radians(theta)
    sin = np.sin(theta)
    
    D = 2 * pi * r / z #刃の間隔
    L = arc_contact(R, d ,T)
    rho = U / (D / sin) #場合分けの変数
    m = math.floor(rho)
    N = math.floor(L / D) + 1 #接触弧上の刃数
    n = N + m #同時切削刃数
    t = (L - ((N - 1) * D)) #変数の定義
    
    l = (m + 1) * (N - 1) * (D / sin) - (N - 1) * (t / sin) + m * (t / sin) + (u / sin) #切削刃長さの計算
    
    return l

In [13]:
def contact_blades_length(R, d, z, theta , T, U):

    pi = np.pi
    r = d / 2
    D = 2 * pi * r / z

    L = arc_contact(R, d ,T)
    N = math.floor(L / D) + 1 #接触弧上の刃数
    t = (L - ((N - 1) * D))
    
    l_max = length_blades_contact(R, d, z, theta, T, U, t)
    l_min = length_blades_contact(R, d, z, theta, T, U, 0)
    
    width = l_max - l_min
    
    theta = np.radians(theta)
    rho = U / (D / np.sin(theta))
    m = math.floor(rho)
    n = N + m
    
    l_max = np.round(l_max, decimals = 2)
    l_min = np.round(l_min, decimals = 2)
    width = np.round(width, decimals = 2)
    
    if width == 0:
        contact_blades_length = print('切削刃長さ', l_max, '(一定)')
        
    else:
        contact_blades_length = pd.DataFrame(
            data = np.array([[l_max, l_min, width, n]]),
            index = ['切削刃長さ'],
            columns = ['最大', '最小', '変化量', '同時切削刃数']
        )
    
    return contact_blades_length

## 2. 直線部の関数

In [14]:
def interval_blades(d, z): #エンドミルの刃の間隔
    pi = np.pi
    r = d / 2
    
    D = 2 * pi * r / z
    
    return D

## 2.1 切込み深さの関数

In [15]:
def depth_cut_straight(d, z, theta): #直線部の半径方向切込み
    theta = np.radians(theta)
    r = d / 2
    D = interval_blades(d, z)
    
    list_ae_s = []
    
    for N in range(2, z):
        ae_s = r * (1 - np.cos(((N - 1) * D) / r))
        list_ae_s.append(ae_s)
    
    if 0 in list_ae_s:
        list_ae_s.remove(0)
    
    ae_s = min(list_ae_s)
    
    ap_s = D / np.tan(theta)
    
    L = r * np.arccos(1 - (ae_s / r))
    n = (L / D) + 1
    
    ae_s = np.round(ae_s, decimals = 2)
    ap_s = np.round(ap_s, decimals = 2)
    n = np.round(n)
    
    depth_cut_s = pd.DataFrame(
        data = np.array([[ae_s, ap_s, n]]),
        index = ['切込み深さ'],
        columns = ['半径方向ae', '軸方向ap', '同時切削刃数']
    )
    
    return depth_cut_s

# コーナー部

### 同時切削刃数, 切削刃長さ：一定のもとで 切込み深さからエンドミル仕様を求める（目標aeと最も近い値をとるものが算出される）

In [16]:
spec_endmill(
    3, #隅R
    6, #エンドミル径
    0.8, #径方向切込み
    3, #軸方向切込み
    10, #最大刃数
)

,刃数,ねじれ角,この仕様でのae,同時切削刃数
エンドミル仕様,8.0,38.1,0.879,4.0


### 同時切削刃数, 切削刃長さ：一定の下でエンドミル仕様から切込み深さを求める(最小値が算出される)

In [34]:
depth_cut(
    3, #隅R
    6, #エンドミル径
    12, #エンドミル刃数
    38, #ねじれ角
    1, #軸方向ap調節(整数のみ)
)

,半径方向ae,軸方向ap,同時切削刃数
切込み深さ,0.4,2.01,5.0


   ### 同時切削刃数：一定の下で 切込み深さから切削刃長さを求める

In [18]:
contact_blades_length(
    3, #隅R
    6, #エンドミル径
    8, #エンドミル刃数
    38, #ねじれ角
    0.02, #半径方向切込み
    5.67, #軸方向切込み
)

,最大,最小,変化量,同時切削刃数
切削刃長さ,15.31,14.75,0.56,4.0


# 直線部

### 同時切削刃数, 切削刃長さ：一定の下でエンドミル仕様から切込み深さを求める(最小値が算出される)

In [41]:
depth_cut_straight(
    6, #エンドミル径
    5, #エンドミル刃数
    38, #ねじれ角
)

,半径方向ae,軸方向ap,同時切削刃数
切込み深さ,2.07,4.83,2.0
